In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from sklearn.ensemble import GradientBoostingRegressor

pd.options.display.max_rows = 20
pd.options.display.max_columns = 100

train_data_file = '../input/train.csv'
type_dictionary = '../input/data_types.csv'
test_data_file = '../input/test.csv'

In [2]:
type_dict = {}
with open(type_dictionary) as f:
    reader = csv.reader(f)
    for row in reader:
        type_dict[row[0]] = row[1]

In [3]:
train = pd.read_csv(train_data_file, dtype=type_dict)
train_raw = train.copy()
train_id = train.Id
train.drop(columns='Id', inplace=True)

In [4]:
test = pd.read_csv(test_data_file, dtype=type_dict)
test_raw = test.copy()
test_id = test.Id
test.drop(columns='Id', inplace=True)

In [5]:
def fill_missing_data(dfin):
    df = dfin.copy()
    categoricals = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
                    'GarageCond','GarageType','GarageFinish','GarageQual',
                   'BsmtExposure','BsmtFinType2','BsmtFinType1','BsmtCond','BsmtQual']
    for column in categoricals:
        df.loc[:,column].cat.add_categories(['None'], inplace=True)
        df.loc[:,column].fillna('None', inplace=True)
    df.MasVnrType.fillna('None', inplace=True)
    
    floats = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath','BsmtUnfSF',
              'GarageArea', 'GarageCars', 'TotalBsmtSF','MasVnrArea']
    for column in floats:
        df.loc[:,column].fillna(0, inplace=True)
        
    df.Exterior1st.fillna('VinylSd', inplace=True)
    df.Exterior2nd.fillna('VinylSd', inplace=True)
    df.Functional.fillna('Typ', inplace=True)
    df.KitchenQual.fillna('TA', inplace=True)
    df.MSZoning.fillna('RL', inplace=True)
    df.SaleType.fillna('WD', inplace=True)
    df.Utilities.fillna('AllPub', inplace=True)
    df.Electrical.fillna('SBrkr', inplace=True)
    
    df.LotFrontage.fillna(df.LotFrontage.median(), inplace=True)
    df.GarageYrBlt.fillna(9999, inplace=True)
    
    return df

In [6]:
def clean_categoricals(dfin):
    df = dfin.copy()
    
    #This will set the proper order for all of the ordinal variables
    df.BsmtCond.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.BsmtExposure.cat.set_categories(['None','No','Mn','Av','Gd'], ordered=True, inplace=True)
    df.BsmtFinType1.cat.set_categories(['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'], ordered=True, inplace=True)
    df.BsmtFinType2.cat.set_categories(['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'], ordered=True, inplace=True)
    df.BsmtQual.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Electrical.cat.set_categories(['Mix','FuseP','FuseF','FuseA','SBrkr'], ordered=True, inplace=True)
    df.ExterCond.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.ExterQual.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Fence.cat.set_categories(['None','MnWw','GdWo','MnPrv','GdPrv'], ordered=True, inplace=True)
    df.FireplaceQu.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Functional.cat.set_categories(['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'], ordered=True, inplace=True)
    df.GarageCond.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.GarageFinish.cat.set_categories(['None','Unf','RFn','Fin'], ordered=True, inplace=True)
    df.GarageQual.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.HeatingQC.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.KitchenQual.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.LandSlope.cat.set_categories(['Sev','Mod','Gtl'], ordered=True, inplace=True)
    df.LotShape.cat.set_categories(['Reg','IR1','IR2','IR3'], ordered=True, inplace=True)
    df.OverallCond.cat.set_categories(['1','2','3','4','5','6','7','8','9','10'], ordered=True, inplace=True)
    df.OverallQual.cat.set_categories(['1','2','3','4','5','6','7','8','9','10'], ordered=True, inplace=True)
    df.PavedDrive.cat.set_categories(['N','P','Y'], ordered=True, inplace=True)
    df.PoolQC.cat.set_categories(['None','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Utilities.cat.set_categories(['ELO','NoSeWa','NoSewr','AllPub'], ordered=True, inplace=True)
    
    # This will set which categories from the nominal variables will be used (NaNs are created)
    df.MSSubClass.cat.set_categories(['20','50','60','Other'], inplace=True)
    df.MSZoning.cat.set_categories(['RL','RM','Other'], inplace=True)
    df.LandContour.cat.set_categories(['Lvl','Other'], inplace=True)
    df.LotConfig.cat.set_categories(['Inside','Other'], inplace=True)
    df.Condition1.cat.set_categories(['Artery', 'Feedr', 'Norm', 'PosA', 'PosN','Other'], inplace=True)
    df.Condition2.cat.set_categories(['Norm','Other'], inplace=True)
    df.BldgType.cat.set_categories(['1Fam','Other'], inplace=True)
    df.HouseStyle.cat.set_categories(['1.5Fin', '1Story', '2Story', 'SLvl', 'Other'], inplace=True)
    df.RoofStyle.cat.set_categories(['Gable','Flat','Hip','Other'], inplace=True)
    df.RoofMatl.cat.set_categories(['CompShg','Other'], inplace=True)
    df.Exterior1st.cat.set_categories(['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd',
       'HdBoard', 'ImStucc', 'MetalSd', 'Plywood', 'Stone', 'Stucco',
       'VinylSd', 'Wd Sdng', 'WdShing'], inplace=True)
    df.Exterior2nd.cat.set_categories(['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd',
       'HdBoard', 'ImStucc', 'MetalSd', 'Plywood', 'Stone', 'Stucco',
       'VinylSd', 'Wd Sdng', 'WdShing'], inplace=True)
    df.Heating.cat.set_categories(['Floor', 'GasA', 'Other'], inplace=True)
    df.GarageType.cat.set_categories(['Attchd','Detchd','None','Other'], inplace=True)
    df.SaleCondition.cat.set_categories(['Normal','Other'], inplace=True)
    
    #This fills the NaNs into the recently created 'Other' category
    others = ['MSSubClass', 'MSZoning', 'LandContour', 'LotConfig', 'Condition1',
              'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
              'Heating', 'GarageType', 'SaleCondition']
    for col in others:
        df[col].fillna('Other', inplace=True)
    
    return df

In [7]:
def all_dummies(df):
    dummies = df.copy()
    ordinals = ['BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtQual',
               'Electrical','ExterCond','ExterQual','Fence','FireplaceQu',
               'Functional','GarageCond','GarageFinish','GarageQual','HeatingQC',
               'KitchenQual','LandSlope','LotShape','OverallCond','OverallQual',
               'PavedDrive','PoolQC','Utilities']
    for col in ordinals:
        dummies[col] = dummies[col].cat.codes
    return pd.get_dummies(dummies, drop_first=True)

In [11]:
def transform_data(dfin):
    df = dfin.copy()
    df_fill = fill_missing_data(df)
    df_ordered = clean_categoricals(df_fill)
    df_drop = df_ordered.drop(['MiscFeature','MiscVal','SaleType','RoofMatl'], axis=1)
    df_dummies = all_dummies(df_drop)
    
    return df_dummies

In [9]:
def fit_predict(regressor, X_train, Y_train, X_test, rname):
    regressor.fit(X_train, Y_train)
    Y_pred = regressor.predict(X_test)
    submission = pd.DataFrame({'Id':test_id.astype(int), 'SalePrice':Y_pred})
    submission.to_csv("../output/" + rname + ".csv", index=False)

In [12]:
X_train = train.iloc[:,:-1]
Y_train = train.iloc[:,-1]
X_train_clean = transform_data(X_train)
test_clean = transform_data(test)

In [13]:
grad_boost_ls = GradientBoostingRegressor(loss='ls', n_estimators=500)
fit_predict(grad_boost_ls, X_train_clean, Y_train, test_clean, "f_grad_boost_ls_500")

In [ ]:
features = pd.DataFrame({'Feature':X_train_clean.columns, 'Importance':grad_boost_ls.feature_importances_})
features.set_index('Feature', inplace=True)
features.sort_index(inplace=True)

In [ ]:
features.loc[features.Importance>0,:].sort_values(by='Importance', ascending=False)

In [ ]:
features.loc[features.Importance==0,:].index